In [26]:
import datetime as dt
from lxml import html
from bs4 import BeautifulSoup
import urllib
import sys
import requests
import operator
from datetime import timedelta
import csv
import pandas as pd
import numpy as np
import math
teams = pd.read_csv('teams_table.csv')
year1617 = pd.read_csv('../GamesData/Strength of Schedule/season1617.csv', parse_dates=['Date'])
year1718 = pd.read_csv('../GamesData/Strength of Schedule/season1718.csv', parse_dates=['Date'])
year1819 = pd.read_csv('../GamesData/Strength of Schedule/season1819.csv', parse_dates=['Date'])
year1920 = pd.read_csv('../GamesData/Strength of Schedule/season1920.csv', parse_dates=['Date'])
curr_metrics = pd.read_csv('1-30-teammetrics.csv')
curr_season = pd.read_csv('1-30-teammetrics.csv')

In [27]:
def fill_games_list(games_table,game_list,yesterday):
    for x in games_table:

        if x.find("tbody").findAll("tr")[0].find("a").has_attr('href') and x.find("tbody").findAll("tr")[1].find("a").has_attr('href'):
            
            try:
                winner = x.find("tbody").find("tr",{"class":"winner"}).find("a").text
                winner = remove_char(winner)
                winner_score = x.find("tbody").find("tr",{"class":"winner"}).find("td",{"class":"right"}).text
                loser = x.find("tbody").find("tr",{"class":"loser"}).find("a").text
                loser = remove_char(loser)
                loser_score = x.find("tbody").find("tr",{"class":"loser"}).find("td",{"class":"right"}).text
                home_team =x.find("tbody").findAll("tr")[1].find("a").text
                home_team = remove_char(home_team)
                away_team =x.find("tbody").findAll("tr")[0].find("a").text
                away_team = remove_char(away_team)
            except:
                continue
            
            if winner in teams['sr_games_name'].unique() and loser in teams['sr_games_name'].unique():
                winnerID = teams[teams['sr_games_name'] == winner]['teamID'].values[0]
                loserID = teams[teams['sr_games_name'] == loser]['teamID'].values[0]
                homeID = teams[teams['sr_games_name'] == home_team]['teamID'].values[0]
                awayID = teams[teams['sr_games_name'] == away_team]['teamID'].values[0]
            else:
                print(winner,loser)
            try:
                game_list.append((yesterday,winnerID,winner_score,loserID,loser_score,homeID,awayID))
            except:
                print((yesterday,winner,winner_score,loser,loser_score,home_team,away_team))
        else:
            pass
        
def get_yest_games():
    game_list = []
    for x in range(150,0,-1):
        yesterday = dt.date.today()- timedelta(days=x)
        if yesterday == dt.date(2020, 1, 20):
            continue
        SR_games = "https://www.sports-reference.com/cbb/boxscores/index.cgi?month="+str(yesterday.month)+"&day="+str(yesterday.day)+"&year="+str(yesterday.year)
        print(SR_games)
#         try:
        games = requests.get(SR_games)
        soup_games = BeautifulSoup(games.content, 'lxml')
        games_table=soup_games.findAll("div",{"class":"game_summary nohover"})
        fill_games_list(games_table,game_list,yesterday)
#         except:
#             print("ERROR!")
    return game_list

def remove_char(teamname):
    if "-" in teamname or "(" in teamname or ")" in teamname or "." in teamname:
        teamname = teamname.replace("-", " ")
        teamname = teamname.replace("(", "")
        teamname = teamname.replace(")", "")
        teamname = teamname.replace(".", "")
    return teamname

In [28]:
# for x in range(150,0,-1):
#         yesterday = dt.date.today()- timedelta(days=x)
#         print(yesterday)

In [29]:
games = get_yest_games()

https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=4&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=5&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=6&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=7&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=8&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=9&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=10&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=11&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=12&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=13&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=14&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=9&day=15&year=2020
https://www.sports-ref

https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=13&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=14&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=15&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=16&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=17&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=18&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=19&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=20&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=21&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=22&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=23&year=2020
https://www.sports-reference.com/cbb/boxscores/index.cgi?month=12&day=24&year=2020
http

In [30]:
curr_season = pd.DataFrame(games,columns=['Date','Winner','WinnerScore','Loser','LoserScore','Home','Away'])
curr_season['WinnerScore'] = pd.to_numeric(curr_season['WinnerScore'])
curr_season['LoserScore'] = pd.to_numeric(curr_season['LoserScore'])
curr_season['Date'] = pd.to_datetime(curr_season['Date'])

In [31]:
curr_season.to_csv('2-1games.csv',index=False)